## Import Packages & Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import scorecardpy as sc

In [2]:
application = pd.read_csv('data/IS453 Group Assignment - Application Data.csv')
bureau = pd.read_csv('data/IS453 Group Assignment - Bureau Data.csv')

In [3]:
print(application.shape)
print(application.columns)
print(bureau.shape)
print(bureau.columns)

(307511, 120)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=120)
(1716428, 17)
Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')


## Get datasets ready to merge

In [4]:
# filter for applicants that do not own a car
application = application[application['FLAG_OWN_CAR'] == 'N']

# filter bureau dataset for CREDIT_CURRENCY = currency 1
bureau = bureau[bureau['CREDIT_CURRENCY'] == 'currency 1']

# drop columns
application = application.drop(columns=['FLAG_OWN_CAR', 'OWN_CAR_AGE', 'NAME_TYPE_SUITE'])
bureau = bureau.drop(columns=['CREDIT_CURRENCY'])

print(application.shape)
print(bureau.shape)

(202924, 117)
(1715020, 16)


In [5]:
bureau['CREDIT_TYPE'].value_counts()

CREDIT_TYPE
Consumer credit                                 1250869
Credit card                                      402071
Car loan                                          27274
Mortgage                                          18295
Microloan                                         12413
Loan for business development                      1954
Another type of loan                               1014
Unknown type of loan                                554
Loan for working capital replenishment              469
Cash loan (non-earmarked)                            56
Real estate loan                                     26
Loan for the purchase of equipment                   19
Loan for purchase of shares (margin lending)          4
Mobile operator loan                                  1
Interbank credit                                      1
Name: count, dtype: int64

In [6]:
agg_bureau = bureau.copy()

# precompute credit status counts
agg_bureau['NUM_ACTIVE_CREDITS'] = (agg_bureau['CREDIT_ACTIVE'] == 'Active').astype(int)
agg_bureau['NUM_CLOSED_CREDITS'] = (agg_bureau['CREDIT_ACTIVE'] == 'Closed').astype(int)
agg_bureau['NUM_BADDEBT_CREDITS'] = (agg_bureau['CREDIT_ACTIVE'] == 'Bad debt').astype(int)

# precompute top 5 most common credit type counts
agg_bureau['NUM_CONSUMER_CREDIT_LOANS'] = (agg_bureau['CREDIT_TYPE'] == 'Consumer credit').astype(int)
agg_bureau['NUM_CREDIT_CARD_LOANS'] = (agg_bureau['CREDIT_TYPE'] == 'Credit card').astype(int)
agg_bureau['NUM_CAR_LOANS'] = (agg_bureau['CREDIT_TYPE'] == 'Car loan').astype(int)
agg_bureau['NUM_MORTGAGE_LOANS'] = (agg_bureau['CREDIT_TYPE'] == 'Mortgage').astype(int)
agg_bureau['NUM_MICRO_LOANS'] = (agg_bureau['CREDIT_TYPE'] == 'Microloan').astype(int)

# DEBT_CREDIT_RATIO = AMT_CREDIT_SUM_DEBT / AMT_CREDIT_SUM
agg_bureau['DEBT_CREDIT_RATIO'] = agg_bureau['AMT_CREDIT_SUM_DEBT'] / agg_bureau['AMT_CREDIT_SUM']

In [7]:
# flatten bureau data
agg_bureau = agg_bureau.groupby('SK_ID_CURR').agg(
    NUM_PREV_LOANS = ('SK_ID_BUREAU', 'count'),
    NUM_ACTIVE_CREDITS = ('NUM_ACTIVE_CREDITS', 'sum'),
    NUM_CLOSED_CREDITS = ('NUM_CLOSED_CREDITS', 'sum'),
    NUM_CONSUMER_CREDIT_LOANS = ('NUM_CONSUMER_CREDIT_LOANS', 'sum'),
    NUM_CREDIT_CARD_LOANS = ('NUM_CREDIT_CARD_LOANS', 'sum'),
    NUM_CAR_LOANS = ('NUM_CAR_LOANS', 'sum'),
    NUM_MORTGAGE_LOANS = ('NUM_MORTGAGE_LOANS', 'sum'),
    NUM_MICRO_LOANS = ('NUM_MICRO_LOANS', 'sum'),
    DAYS_CREDIT_MIN = ('DAYS_CREDIT', 'min'), # capture the oldest credit application
    DAYS_CREDIT_MAX = ('DAYS_CREDIT', 'max'), # find most recent credit application
    DAYS_CREDIT_MEAN = ('DAYS_CREDIT', 'mean'), # avg days before current application, for prev credit lines
    DAYS_CREDIT_OVERDUE_MAX = ('CREDIT_DAY_OVERDUE', 'max'),
    DAYS_CREDIT_OVERDUE_MEAN = ('CREDIT_DAY_OVERDUE', 'mean'),
    DAYS_CREDIT_ENDDATE_MEAN = ('DAYS_CREDIT_ENDDATE', 'mean'),
    CNT_CREDIT_PROLONG_MAX = ('CNT_CREDIT_PROLONG', 'max'),
    CNT_CREDIT_PROLONG_MEAN = ('CNT_CREDIT_PROLONG', 'mean'),
    AMT_CREDIT_SUM_LIMIT_MEAN = ('AMT_CREDIT_SUM_LIMIT', 'mean'),
    AMT_CREDIT_SUM_OVERDUE_SUM = ('AMT_CREDIT_SUM_OVERDUE', 'sum'),
    AMT_CREDIT_SUM_OVERDUE_MAX = ('AMT_CREDIT_SUM_OVERDUE', 'max'),
    DAYS_CREDIT_UPDATE_MEAN = ('DAYS_CREDIT_UPDATE', 'mean'),
    DEBT_CREDIT_RATIO_MEAN = ('DEBT_CREDIT_RATIO', 'mean'),
    AMT_ANNUITY_MEAN = ('AMT_ANNUITY', 'mean'),
).reset_index()

## Merge Datasets

In [8]:
print(application.shape)
print(bureau.shape)
print(agg_bureau.shape)

(202924, 117)
(1715020, 16)
(305773, 23)


In [9]:
merged = pd.merge(left=application, right=agg_bureau, on='SK_ID_CURR', how='left')
print(merged.shape)

(202924, 139)


## Calculate IVs for each x variable

In [10]:
# get target & independent variables
cols = merged.columns.to_list()[1:]
print(cols)

merged_working = merged.copy()

['TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENT

In [12]:
# unable to calculate WOE & IV for ORGANIZATION_TYPE as it is because it has too many possible categories
    # group some values together

mapping = {
    'Business Entity Type 1': 'Business Entity',
    'Business Entity Type 2': 'Business Entity',
    'Business Entity Type 3': 'Business Entity',
    'Trade: type 7': 'Trade',
    'Trade: type 3': 'Trade',
    'Trade: type 2': 'Trade',
    'Trade: type 6': 'Trade',
    'Trade: type 1': 'Trade',
    'Trade: type 4': 'Trade',
    'Trade: type 5': 'Trade',
    'Transport: type 1': 'Transport',
    'Transport: type 2': 'Transport',
    'Transport: type 3': 'Transport',
    'Transport: type 4': 'Transport',
    'Industry: type 3': 'Industry',
    'Industry: type 11': 'Industry',
    'Industry: type 9': 'Industry',
    'Industry: type 7': 'Industry',
    'Industry: type 1': 'Industry',
    'Industry: type 4': 'Industry',
    'Industry: type 5': 'Industry',
    'Industry: type 6': 'Industry',
    'Industry: type 2': 'Industry',
    'Industry: type 10': 'Industry',
    'Industry: type 12': 'Industry',
    'Industry: type 13': 'Industry',
    'Industry: type 8': 'Industry',
    'Government': 'Public Sector',
    'Transport': 'Public Sector',
    'Military': 'Public Sector',
    'Police': 'Public Sector',
    'Hotel': 'Hospitality',
    'Restaurant': 'Hospitality',
    'Bank': 'Financial Services',
    'Insurance': 'Financial Services',
}

merged_working['ORGANIZATION_TYPE'] = merged_working['ORGANIZATION_TYPE'].replace(mapping)
print(merged_working['ORGANIZATION_TYPE'].value_counts())

ORGANIZATION_TYPE
Business Entity        50934
XNA                    45205
Self-employed          23916
Public Sector          13682
Other                  10739
Trade                   9680
Industry                8752
Medicine                8270
School                  6471
Kindergarten            5404
Construction            3445
Hospitality             2052
Financial Services      1985
Security                1829
Housing                 1817
Postal                  1684
Agriculture             1450
Services                1122
Security Ministries     1109
University               901
Electricity              568
Telecom                  340
Culture                  271
Advertising              269
Emergency                242
Realtor                  211
Cleaning                 195
Mobile                   185
Legal Services           145
Religion                  51
Name: count, dtype: int64


In [14]:
merged_working = merged_working.loc[:, cols]

# automatically calculate bin ranges, WOE, and IV for the independent variables
bins = sc.woebin(merged_working, y='TARGET')

# scorecardpy calcualtes WoE as the opposite of the normal formula
for variable, bindetails in bins.items():
    bins[variable]['woe'] = bins[variable]['woe'] * -1

# print bin details and IV
for variable, bindetails in bins.items():
    print(variable , ": ")
    print('IV: ' + str(round(bindetails['total_iv'][0],4)))
    display(bindetails)
    print("--"*50)

[INFO] creating woe binning ...


/Users/caraloo/opt/anaconda3/envs/app_env/lib/python3.9/site-packages/scorecardpy/woebin.py:320: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  init_bin = dtm.groupby('bin', group_keys=False)['y'].agg([n0, n1])\
/Users/caraloo/opt/anaconda3/envs/app_env/lib/python3.9/site-packages/scorecardpy/woebin.py:320: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  init_bin = dtm.groupby('bin', group_keys=False)['y'].agg([n0, n1])\
/Users/caraloo/opt/anaconda3/envs/app_env/lib/python3.9/site-packages/scorecardpy/woebin.py:136: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum

Binning on 202924 rows and 136 columns in 00:02:46
FLAG_DOCUMENT_5 : 
IV: 0.0


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLAG_DOCUMENT_5,"[-inf,inf)",202924,1.0,185675,17249,0.085002,-0.0,0.0,0.0,inf,False


----------------------------------------------------------------------------------------------------
NONLIVINGAREA_MODE : 
IV: 0.0236


...

----------------------------------------------------------------------------------------------------
FLAG_DOCUMENT_11 : 
IV: 0.0


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLAG_DOCUMENT_11,"[-inf,inf)",202924,1.0,185675,17249,0.085002,-0.0,0.0,0.0,inf,False


----------------------------------------------------------------------------------------------------
REG_REGION_NOT_LIVE_REGION : 
IV: 0.0


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,REG_REGION_NOT_LIVE_REGION,"[-inf,inf)",202924,1.0,185675,17249,0.085002,-0.0,0.0,0.0,inf,False


----------------------------------------------------------------------------------------------------
